## Project 1 
#### Imports:

In [40]:
import numpy as np
from implementations import *
from proj1_helpers import *
from cross_validation import *
from pre_processing import *
from split_jet_num import generate_4_sets_looking_on_jetnum, columns_contains_just_missing_values, columns_contains_same_value

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load data set

In [2]:
""" y: class labels
    tx: features
    ids: event ids """
y, tx, ids = load_csv_data("datas/train.csv", sub_sample=True)
print("TRAIN DATAS LOADED!")

no_y, tx_test, ids_test = load_csv_data("datas/test.csv", sub_sample=True)
print("TEST DATAS LOADED!")

TRAIN DATAS LOADED!
TEST DATAS LOADED!


## Preprocessing 
### Replace missing values with mean, median  or normalize the dataset

In [379]:
# Replace missing values with mean for each feature
# train set
means = find_mean(tx)
tx_replaced_by_mean = replace_missing_values(tx, means)

# test set
means_test = find_mean(tx_test)
tx_replaced_by_mean_test = replace_missing_values(tx_test, means_test)

In [380]:
# Replace missing values with median for each feature
# train set
medians = find_median(tx)
tx_replaced_by_median = replace_missing_values(tx, medians)

# test set
medians_test = find_median(tx_test)
tx_replaced_by_median_test = replace_missing_values(tx_test, medians_test)

In [381]:
# Replace missing values with 0 and before that normalize all values without considering missing values
std_data_tx_with_mask = standardize(clean_array(tx))
tx_std_data_replaced_by_0 = replace_missing_values(std_data_tx_with_mask, np.full((30, 1), 0))

## Division of the dataset looking on jet num
If PRI_jet_num is zero or one then some features are -999.
Divide dataset in 4 looking on jet_num 0, 1, 2 and 3.

In [3]:
features_jet_0, features_jet_1, features_jet_2, features_jet_3, y_jet_0, y_jet_1, y_jet_2, y_jet_3, ids_jet_0, ids_jet_1, ids_jet_2, ids_jet_3 = generate_4_sets_looking_on_jetnum(tx, y, ids)

For each set look how many missing values there are.. in order to detect how many features we want to drop!

In [4]:
# iterate to find which columns to drop
columns_to_remove_0 = columns_contains_just_missing_values(features_jet_0[0])
columns_to_remove_1 = columns_contains_just_missing_values(features_jet_1[0])
columns_to_remove_2 = columns_contains_just_missing_values(features_jet_2[0])
columns_to_remove_3 = columns_contains_just_missing_values(features_jet_3[0])

We have noticed that we don't need to remove any features when jet num is 2 and 3. 


Infact: 


JET_NUM = 0 -> [4, 5, 6, 12, 23, 24, 25, 26, 27, 28]


JET_NUM = 1 -> [4, 5, 6, 12, 26, 27, 28]


JET_NUM = 2 -> []


JET_NUM = 3 -> []


We will drop features: 4, 5, 6, 12, 26, 27 and 28. And also we will drop feature 22 since it is the one of jet_num.

In [5]:
# drop feature 22 since it is the one of jet_num and it will contains the same value.
columns_to_remove_0.append(22)
columns_to_remove_1.append(22)
columns_to_remove_2.append(22)
columns_to_remove_3.append(22)

Check for constant values, if I feature contains all the same values it is not important.

In [6]:
columns_to_remove_0_b = columns_contains_same_value(features_jet_0[0])
columns_to_remove_1_b = columns_contains_same_value(features_jet_1[0])
columns_to_remove_2_b = columns_contains_same_value(features_jet_2[0])
columns_to_remove_3_b = columns_contains_same_value(features_jet_3[0])

We can notice that as predicted we should removed feature 22. We have found also feature 29 to remove from column 0.
Since before we have dropped columns full of -999 (same value) we can just remove these features from each set.

JET_NUM = 0 -> [4, 5, 6, 12, 22, 23, 24, 25, 26, 27, 28, 29]


JET_NUM = 1 -> [4, 5, 6, 12, 22, 26, 27, 28]


JET_NUM = 2 -> [22]


JET_NUM = 3 -> [22]

In [7]:
# remove columns from subset    
features_dropped_0 = np.delete(features_jet_0[0], columns_to_remove_0_b, axis=1)
features_dropped_1 = np.delete(features_jet_1[0], columns_to_remove_1_b, axis=1)
features_dropped_2 = np.delete(features_jet_2[0], columns_to_remove_2_b, axis=1)
features_dropped_3 = np.delete(features_jet_3[0], columns_to_remove_3_b, axis=1)

In [8]:
columns_to_remove = [4, 5, 6, 12, 22, 26, 27, 28]

tx_dropped_columns = np.delete(tx, columns_to_remove, axis=1)

## Cross validation 
K-fold cross-validation: original sample randomly partitioned into k equal sized subsamples.
Repeated k times.

In [9]:
seed = 19
degree = 7
k_fold = 5

lambdas = np.logspace(-4, 0, 30) #just for ridge regression

# split data in k fold
k_indices = build_k_indices(y, k_fold, seed)

## Test ML Methods
### Least Squares

#### Results obtained using k = 5 and replacing missing value by mean value.

In [385]:
# store the accuracy of training data and test data
accuracy_train = []
accuracy_test = []
losses = []

for k in range(k_fold):
    loss, w, single_accuracy_train, single_accuracy_test = cross_validation(y, tx_replaced_by_mean, k_indices, k, degree, least_squares)
    accuracy_train.append(single_accuracy_train)
    accuracy_test.append(single_accuracy_test)
    losses.append(loss)
    
# Just for study the behaviour
n = len(accuracy_train)
for i in range(n):
    print("Iteration: %d) Accuracy train: %f - Accuracy test: %f - Loss: %f\n" % (i, accuracy_train[i], accuracy_test[i], losses[i]))

mean_accuracy_test = np.mean(accuracy_test)
min_accuracy_test = np.min(accuracy_test)
max_accuracy_test = np.max(accuracy_test)

mean_accuracy_train = np.mean(accuracy_train)
min_accuracy_train = np.min(accuracy_train)
max_accuracy_train = np.max(accuracy_train)

print("Accuracy test, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_test, min_accuracy_test, max_accuracy_test))
print("Accuracy train, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_train, min_accuracy_train, max_accuracy_train))

Iteration: 0) Accuracy train: 0.804520 - Accuracy test: 0.799740 - Loss: 0.575760

Iteration: 1) Accuracy train: 0.800310 - Accuracy test: 0.802020 - Loss: 0.585639

Iteration: 2) Accuracy train: 0.806710 - Accuracy test: 0.808080 - Loss: 0.572250

Iteration: 3) Accuracy train: 0.804575 - Accuracy test: 0.800720 - Loss: 0.576161

Iteration: 4) Accuracy train: 0.803565 - Accuracy test: 0.804200 - Loss: 0.577485

Accuracy test, mean: 0.802952, min value: 0.799740, max value: 0.808080 

Accuracy train, mean: 0.803936, min value: 0.800310, max value: 0.806710 



#### Results obtained using k = 5 and replacing missing value by median value.

In [386]:
# store the accuracy of training data and test data
accuracy_train = []
accuracy_test = []
losses = []
weights = []

for k in range(k_fold):
    loss, w, single_accuracy_train, single_accuracy_test = cross_validation(y, tx_replaced_by_median, k_indices, k, degree, least_squares)
    accuracy_train.append(single_accuracy_train)
    accuracy_test.append(single_accuracy_test)
    losses.append(loss)
    weights.append(w)

# Just for study the behaviour    
n = len(accuracy_train)
for i in range(n):
    print("Iteration: %d) Accuracy train: %f - Accuracy test: %f - Loss: %f\n" % (i, accuracy_train[i], accuracy_test[i], losses[i]))

mean_accuracy_test = np.mean(accuracy_test)
min_accuracy_test = np.min(accuracy_test)
max_accuracy_test = np.max(accuracy_test)

mean_accuracy_train = np.mean(accuracy_train)
min_accuracy_train = np.min(accuracy_train)
max_accuracy_train = np.max(accuracy_train)

print("Accuracy test, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_test, min_accuracy_test, max_accuracy_test))
print("Accuracy train, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_train, min_accuracy_train, max_accuracy_train))


Iteration: 0) Accuracy train: 0.805525 - Accuracy test: 0.801640 - Loss: 0.573839

Iteration: 1) Accuracy train: 0.804210 - Accuracy test: 0.807480 - Loss: 0.576331

Iteration: 2) Accuracy train: 0.808065 - Accuracy test: 0.809400 - Loss: 0.569957

Iteration: 3) Accuracy train: 0.805485 - Accuracy test: 0.801400 - Loss: 0.574242

Iteration: 4) Accuracy train: 0.804755 - Accuracy test: 0.805380 - Loss: 0.575564

Accuracy test, mean: 0.805060, min value: 0.801400, max value: 0.809400 

Accuracy train, mean: 0.805608, min value: 0.804210, max value: 0.808065 



#### Results obtained using k = 5 and replacing missing value by 0 after having normalize

In [ ]:
accuracy_train = []
accuracy_test = []
losses = []
weights = []


for k in range(k_fold):
    loss, w, single_accuracy_train, single_accuracy_test = cross_validation(y, tx_std_data_replaced_by_0, k_indices, k, degree, least_squares)
    accuracy_train.append(single_accuracy_train)
    accuracy_test.append(single_accuracy_test)
    losses.append(loss)
    weights.append(w)

# Just for study    
n = len(accuracy_train)
for i in range(n):
    print("Iteration: %d) Accuracy train: %f - Accuracy test: %f - Loss: %f\n" % (i, accuracy_train[i], accuracy_test[i], losses[i]))

mean_accuracy_test = np.mean(accuracy_test)
min_accuracy_test = np.min(accuracy_test)
max_accuracy_test = np.max(accuracy_test)

mean_accuracy_train = np.mean(accuracy_train)
min_accuracy_train = np.min(accuracy_train)
max_accuracy_train = np.max(accuracy_train)

print("Accuracy test, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_test, min_accuracy_test, max_accuracy_test))
print("Accuracy train, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_train, min_accuracy_train, max_accuracy_train))


### Ridge regression using normal equations

#### Results obtained using k = 5 and replacing missing value by mean value.

In [ ]:
accuracy_train = []
accuracy_test = []
losses = []

for lambda_ in lambdas:
    accuracy_train_temp = []
    accuracy_test_temp = []
    losses_temp = []
    for k in range(k_fold):
        loss, w, single_accuracy_train, single_accuracy_test = cross_validation(y, tx_replaced_by_mean, k_indices, k, degree, ridge_regression, lambda_=lambda_)
        accuracy_train_temp.append(single_accuracy_train)
        accuracy_test_temp.append(single_accuracy_test)
        losses_temp.append(loss)
    accuracy_train.append(np.mean(accuracy_train_temp))
    accuracy_test.append(np.mean(accuracy_test_temp))
    losses.append(np.mean(losses_temp))

# Just for study    
n = len(accuracy_train)
for i in range(n):
    print("Iteration: %d) Accuracy train: %f - Accuracy test: %f - Loss: %f\n" % (i, accuracy_train[i], accuracy_test[i], losses[i]))

mean_accuracy_test = np.mean(accuracy_test)
min_accuracy_test = np.min(accuracy_test)
max_accuracy_test = np.max(accuracy_test)

mean_accuracy_train = np.mean(accuracy_train)
min_accuracy_train = np.min(accuracy_train)
max_accuracy_train = np.max(accuracy_train)

print("Accuracy test, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_test, min_accuracy_test, max_accuracy_test))
print("Accuracy train, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_train, min_accuracy_train, max_accuracy_train))

#### Results obtained using k = 5 and replacing missing value by median value.

In [ ]:
accuracy_train = []
accuracy_test = []
losses = []


accuracy_train = []
accuracy_test = []
losses = []
for lambda_ in lambdas:
    accuracy_train_temp = []
    accuracy_test_temp = []
    losses_temp = []
    for k in range(k_fold):
        loss, w, single_accuracy_train, single_accuracy_test = cross_validation(y, tx_replaced_by_median, k_indices, k, degree, ridge_regression, lambda_=lambda_)
        accuracy_train_temp.append(single_accuracy_train)
        accuracy_test_temp.append(single_accuracy_test)
        losses_temp.append(loss)
    accuracy_train.append(np.mean(accuracy_train_temp))
    accuracy_test.append(np.mean(accuracy_test_temp))
    losses.append(np.mean(losses_temp))
    
n = len(accuracy_train)
for i in range(n):
    print("Iteration: %d) Accuracy train: %f - Accuracy test: %f - Loss: %f\n" % (i, accuracy_train[i], accuracy_test[i], losses[i]))

mean_accuracy_test = np.mean(accuracy_test)
min_accuracy_test = np.min(accuracy_test)
max_accuracy_test = np.max(accuracy_test)

mean_accuracy_train = np.mean(accuracy_train)
min_accuracy_train = np.min(accuracy_train)
max_accuracy_train = np.max(accuracy_train)

print("Accuracy test, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_test, min_accuracy_test, max_accuracy_test))
print("Accuracy train, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_train, min_accuracy_train, max_accuracy_train))

## Test cross validation, division by jet_num with different ML implementations

Test jet_num_0 dataset with:
- **Least squares regression using normal equations**: <br/>
    Accuracy test, mean: 0.583375, min value: 0.511335, max value: 0.795970 <br/>
    Accuracy train, mean: 0.586020, min value: 0.511335, max value: 0.773929 
- **Ridge regression using normal equations**: <br/>
    Accuracy test, mean: 0.503224, min value: 0.476574, max value: 0.531486 <br/>
    Accuracy train, mean: 0.503237, min value: 0.473804, max value: 0.538035 
- **Linear regression using gradient descent**: <br/>
- **Linear regression using stochastic gradient descent**:
- **Logistic regression using gradient descent or SGD**:
- **Regularized logistic regression using gradient descent or SGD**:

In [10]:
# accuracy_train = []
# accuracy_test = []
# losses = []
# weights = []

# for k in range(k_fold):
#     loss, w, single_accuracy_train, single_accuracy_test = cross_validation(y, tx_dropped, k_indices, k, degree, least_squares)
#     accuracy_train.append(single_accuracy_train)
#     accuracy_test.append(single_accuracy_test)
#     losses.append(loss)
#     weights.append(w)

    
# n = len(accuracy_train)
# for i in range(n):
#     print("Iteration: %d) Accuracy train: %f - Accuracy test: %f - Loss: %f\n" % (i, accuracy_train[i], accuracy_test[i], losses[i]))

# mean_accuracy_test = np.mean(accuracy_test)
# min_accuracy_test = np.min(accuracy_test)
# max_accuracy_test = np.max(accuracy_test)

# mean_accuracy_train = np.mean(accuracy_train)
# min_accuracy_train = np.min(accuracy_train)
# max_accuracy_train = np.max(accuracy_train)

# print("Accuracy test, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_test, min_accuracy_test, max_accuracy_test))
# print("Accuracy train, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_train, min_accuracy_train, max_accuracy_train))

#LEAST SQUARE
accuracy_train = []
accuracy_test = []
losses = []
weights = []

k_indices_0 = build_k_indices(y_jet_0, k_fold, seed)

for k in range(k_fold):
    loss, w, single_accuracy_train, single_accuracy_test = cross_validation(y_jet_0, features_dropped_0, k_indices_0, k, degree, least_squares)
    accuracy_train.append(single_accuracy_train)
    accuracy_test.append(single_accuracy_test)
    losses.append(loss)
    weights.append(w)

    
n = len(accuracy_train)
for i in range(n):
    print("Iteration: %d) Accuracy train: %f - Accuracy test: %f - Loss: %f\n" % (i, accuracy_train[i], accuracy_test[i], losses[i]))

mean_accuracy_test = np.mean(accuracy_test)
min_accuracy_test = np.min(accuracy_test)
max_accuracy_test = np.max(accuracy_test)

mean_accuracy_train = np.mean(accuracy_train)
min_accuracy_train = np.min(accuracy_train)
max_accuracy_train = np.max(accuracy_train)

print("Accuracy test, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_test, min_accuracy_test, max_accuracy_test))
print("Accuracy train, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_train, min_accuracy_train, max_accuracy_train))

Iteration: 0) Accuracy train: 0.511335 - Accuracy test: 0.513854 - Loss: 417040.188757

Iteration: 1) Accuracy train: 0.773929 - Accuracy test: 0.795970 - Loss: 0.739157

Iteration: 2) Accuracy train: 0.529597 - Accuracy test: 0.511335 - Loss: 95905.419265

Iteration: 3) Accuracy train: 0.534635 - Accuracy test: 0.549118 - Loss: 1306.744284

Iteration: 4) Accuracy train: 0.580605 - Accuracy test: 0.546599 - Loss: 21577866.290756

Accuracy test, mean: 0.583375, min value: 0.511335, max value: 0.795970 

Accuracy train, mean: 0.586020, min value: 0.511335, max value: 0.773929 



/Users/costanzavolpini/Git/project1-ML/scripts/implementations.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w = np.linalg.lstsq(a, b)[0]


In [57]:
# RIDGE REGRESSION
accuracy_train = []
accuracy_test = []
losses = []

for lambda_ in lambdas:
    accuracy_train_temp = []
    accuracy_test_temp = []
    losses_temp = []
    for k in range(k_fold):
        loss, w, single_accuracy_train, single_accuracy_test = cross_validation(y_jet_0, features_dropped_0, k_indices_0, k, degree, ridge_regression, lambda_=lambda_)
        accuracy_train_temp.append(single_accuracy_train)
        accuracy_test_temp.append(single_accuracy_test)
        losses_temp.append(loss)
    accuracy_train.append(np.mean(accuracy_train_temp))
    accuracy_test.append(np.mean(accuracy_test_temp))
    losses.append(np.mean(losses_temp))

# Just for study    
n = len(accuracy_train)
for i in range(n):
    print("Iteration: %d) Accuracy train: %f - Accuracy test: %f - Loss: %f\n" % (i, accuracy_train[i], accuracy_test[i], losses[i]))

mean_accuracy_test = np.mean(accuracy_test)
min_accuracy_test = np.min(accuracy_test)
max_accuracy_test = np.max(accuracy_test)

mean_accuracy_train = np.mean(accuracy_train)
min_accuracy_train = np.min(accuracy_train)
max_accuracy_train = np.max(accuracy_train)

print("Accuracy test, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_test, min_accuracy_test, max_accuracy_test))
print("Accuracy train, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_train, min_accuracy_train, max_accuracy_train))


Iteration: 0) Accuracy train: 0.506423 - Accuracy test: 0.504786 - Loss: 249676896.542092

Iteration: 1) Accuracy train: 0.473804 - Accuracy test: 0.481612 - Loss: 268042139218.589996

Iteration: 2) Accuracy train: 0.499748 - Accuracy test: 0.504786 - Loss: 373696623.679606

Iteration: 3) Accuracy train: 0.495340 - Accuracy test: 0.510327 - Loss: 839357806.641875

Iteration: 4) Accuracy train: 0.518388 - Accuracy test: 0.503778 - Loss: 6229035830927.332031

Iteration: 5) Accuracy train: 0.523804 - Accuracy test: 0.501763 - Loss: 612586082.687964

Iteration: 6) Accuracy train: 0.525819 - Accuracy test: 0.499748 - Loss: 189221295.862311

Iteration: 7) Accuracy train: 0.522796 - Accuracy test: 0.530982 - Loss: 12777318717.023151

Iteration: 8) Accuracy train: 0.480227 - Accuracy test: 0.499244 - Loss: 14944530495.687542

Iteration: 9) Accuracy train: 0.483249 - Accuracy test: 0.494710 - Loss: 6632744315.562798

Iteration: 10) Accuracy train: 0.511461 - Accuracy test: 0.496222 - Loss: 2910

In [90]:
# GRADIENT DESCEND 
# HEREEEE!
accuracy_train = []
accuracy_test = []
losses = []
weights = []
gamma = 0.01
max_iterations = 500

# CHECK SHAPES!!!!!!!!!!!!!!!!!!!!! and correct!
k_indices_0 = build_k_indices(y_jet_0, k_fold, seed)

initial_w = np.zeros(features_dropped_0.shape[1])

for k in range(k_fold):
    loss, w, single_accuracy_train, single_accuracy_test = cross_validation(y_jet_0, features_dropped_0, k_indices_0, k, degree, gradient_descent, initial_w=initial_w, max_iters=max_iterations, gamma=gamma)
    accuracy_train.append(single_accuracy_train)
    accuracy_test.append(single_accuracy_test)
    losses.append(loss)
    weights.append(w)

n = len(accuracy_train)
for i in range(n):
    print("Iteration: %d) Accuracy train: %f - Accuracy test: %f - Loss: %f\n" % (i, accuracy_train[i], accuracy_test[i], losses[i]))

mean_accuracy_test = np.mean(accuracy_test)
min_accuracy_test = np.min(accuracy_test)
max_accuracy_test = np.max(accuracy_test)

mean_accuracy_train = np.mean(accuracy_train)
min_accuracy_train = np.min(accuracy_train)
max_accuracy_train = np.max(accuracy_train)

print("Accuracy test, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_test, min_accuracy_test, max_accuracy_test))
print("Accuracy train, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_train, min_accuracy_train, max_accuracy_train))

ValueError: shapes (1588,127) and (18,) not aligned: 127 (dim 1) != 18 (dim 0)

### Generate Prediction (example)

In [ ]:
y_test_predicted = []
test_poly = build_poly(tx_median_test, degree)
y_test_predicted = predict_labels(weights[0], test_poly)
create_csv_submission(ids_test, y_test_predicted, "submission-7")

In [80]:
ws = [init_w]
ws

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.])]

In [50]:
y_jet_0.shape

(1985,)

In [39]:
tx.shape

(5000, 30)

In [43]:
y.shape

(5000,)